# Validasi Data dengan Great Expectations
 
**Nama:** Hafiz Alfariz

Notebook ini digunakan untuk melakukan validasi data hasil cleaning (`P2M3_hafiz_alfariz_data_clean.csv`) menggunakan Great Expectations dengan Data Context, Validator, dan Expectation Suite.
 


# 1. Import Library

In [1]:
# Import library 
import pandas as pd
import great_expectations as ge
from great_expectations.data_context import FileDataContext

# 2. Setup Data Contect.

In [2]:
# Setup Data Context untuk Great Expectations
context = FileDataContext.create(project_root_dir='./gx_context') 

# 3. Setup Data Source dan Data Aseet.

In [3]:
# Setup Datasource dan Data Asset dari file CSV hasil cleaning

datasource_name = 'supermarket_sales'
datasource = context.sources.add_pandas(datasource_name)
asset_name = 'supermarket_sales_clean'
csv_file = 'P2M3_hafiz_alfariz_data_clean.csv'
asset = datasource.add_csv_asset(asset_name, filepath_or_buffer=csv_file)
batch_request = asset.build_batch_request()


# 4. Create Expectation Suite & Validator

In [4]:
# Membuat Expectation Suite dan Validator
expectation_suite_name = 'supermarket_clean_suite'
context.add_or_update_expectation_suite(expectation_suite_name)
validator = context.get_validator(
    batch_request=batch_request,
    expectation_suite_name=expectation_suite_name
    )
validator.head()

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

,invoice_id,branch,city,customer_type,gender,product_line,unit_price,quantity,tax_5pct,sales,date,time,payment,cogs,gross_margin_percentage,gross_income,rating
0,750-67-8428,Alex,Yangon,Member,Female,Health and beauty,74.69,7,26.14,548.97,2019-01-05,NaN,Ewallet,522.83,4.76,26.14,9.1
1,226-31-3081,Giza,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.82,80.22,2019-03-08,NaN,Cash,76.40,4.76,3.82,9.6
2,631-41-3108,Alex,Yangon,Normal,Female,Home and lifestyle,46.33,7,16.22,340.53,2019-03-03,NaN,Credit card,324.31,4.76,16.22,7.4
3,123-19-1176,Alex,Yangon,Member,Female,Health and beauty,58.22,8,23.29,489.05,2019-01-27,NaN,Ewallet,465.76,4.76,23.29,8.4
4,373-73-7910,Alex,Yangon,Member,Female,Sports and travel,86.31,7,30.21,634.38,2019-02-08,NaN,Ewallet,604.17,4.76,30.21,5.3


# 5. Expectation

## 5.1 Expectation 1: Kolom `invoice_id` harus unik

Kolom invoice_id digunakan sebagai identitas unik setiap transaksi di supermarket. Validasi ini memastikan tidak ada duplikasi invoice, sehingga setiap transaksi dapat ditelusuri dan dianalisis secara akurat. Keunikan invoice_id sangat penting untuk integritas data, pelacakan penjualan, dan audit.

In [5]:
# Penjelasan: Setiap transaksi harus memiliki invoice_id yang unik.
result_unique = validator.expect_column_values_to_be_unique('invoice_id')
print('Success:', result_unique['success'])
result_unique

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

Success: True


{
  "success": true,
  "result": {
    "element_count": 1000,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

**Analisa/summary hasil validasi:**
- Jumlah data: 1000 baris
- Jumlah duplikat: 0
- Persentase duplikat: 0%
- Tidak ada nilai yang hilang (missing)
- Semua nilai pada kolom `invoice_id` adalah unik

Hasil ini menunjukkan data transaksi sudah terstruktur dengan baik dan tidak ada masalah duplikasi pada invoice_id. Data siap digunakan untuk analisis lebih lanjut dan dapat diandalkan untuk proses bisnis maupun audit.

## 5.2 Expectation 2: Kolom `rating` harus berada di antara 0 dan 10

Kolom `rating` merepresentasikan penilaian pelanggan terhadap transaksi di supermarket, dengan skala 0 hingga 10. Validasi ini memastikan seluruh nilai rating berada dalam rentang yang logis dan sesuai standar, sehingga analisis kepuasan pelanggan dapat dilakukan secara akurat.

In [6]:
# Penjelasan: Nilai rating transaksi harus valid antara 0 dan 10.
result_range = validator.expect_column_values_to_be_between('rating', min_value=0, max_value=10)
print('Success:', result_range['success'])
result_range

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

Success: True


{
  "success": true,
  "result": {
    "element_count": 1000,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

**Analisa/summary hasil validasi:**
- Jumlah data: 1000 baris
- Jumlah nilai di luar rentang 0-10: 0
- Persentase nilai tidak valid: 0%
- Tidak ada nilai yang hilang (missing)
- Semua nilai rating berada di antara 0 dan 10

Hasil ini menunjukkan data rating sudah bersih dan valid, sehingga dapat digunakan untuk analisis kepuasan pelanggan, evaluasi layanan, dan pengambilan keputusan bisnis tanpa risiko bias akibat data yang tidak sesuai.

## 5.3 Expectation 3: Kolom `gender` hanya berisi `Male` atau `Female`

Kolom `gender` merepresentasikan jenis kelamin pelanggan yang melakukan transaksi di supermarket, dengan nilai yang diharapkan hanya 'Male' atau 'Female'. Validasi ini penting untuk memastikan data demografi pelanggan konsisten dan tidak ada nilai yang salah atau di luar standar, sehingga analisis segmentasi dan perilaku pelanggan dapat dilakukan secara akurat.


In [7]:
# Penjelasan: Gender pelanggan harus valid, hanya 'Male' atau 'Female'.
result_in_set = validator.expect_column_values_to_be_in_set('gender', ['Male', 'Female'])
print('Success:', result_in_set['success'])
result_in_set

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

Success: True


{
  "success": true,
  "result": {
    "element_count": 1000,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

**Analisa/summary hasil validasi:**
- Jumlah data: 1000 baris
- Jumlah nilai di luar 'Male' atau 'Female': 0
- Persentase nilai tidak valid: 0%
- Tidak ada nilai yang hilang (missing)
- Semua nilai pada kolom `gender` valid

Hasil ini menunjukkan data gender pelanggan sudah bersih dan konsisten, sehingga dapat digunakan untuk analisis demografi, segmentasi pasar, dan pengambilan keputusan bisnis tanpa risiko bias akibat data yang tidak sesuai.

## 5.4 Expectation 4: Kolom `quantity` harus bertipe integer

Kolom `quantity` menunjukkan jumlah barang yang dibeli pada setiap transaksi. Validasi tipe data integer sangat penting agar data dapat digunakan untuk perhitungan total penjualan, analisis stok, dan agregasi statistik tanpa risiko error akibat tipe data yang tidak sesuai.


In [8]:
# Penjelasan: Jumlah barang yang dibeli harus bertipe integer.
result_type = validator.expect_column_values_to_be_of_type('quantity', 'int64')
print('Success:', result_type['success'])
result_type

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

Success: True


{
  "success": true,
  "result": {
    "observed_value": "int64"
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

**Analisa/summary hasil validasi:**
- Semua nilai pada kolom `quantity` bertipe `int64` (integer)
- Tidak ditemukan nilai dengan tipe data lain
- Tidak ada error atau exception selama validasi

Hasil ini memastikan data jumlah barang sudah konsisten dan siap digunakan untuk analisis penjualan, forecasting, dan pengelolaan inventori secara akurat.

## 5.5 Expectation 5: Kolom `payment` hanya boleh berisi [`Cash`, `Credit card`, `Ewallet`]

Kolom `payment` menunjukkan metode pembayaran yang digunakan pelanggan saat bertransaksi di supermarket, yaitu 'Cash', 'Credit card', atau 'Ewallet'. Validasi ini memastikan seluruh data pembayaran konsisten dan tidak ada nilai di luar tiga metode yang diizinkan, sehingga analisis preferensi pembayaran dan pengelolaan transaksi dapat dilakukan dengan akurat.


In [9]:
# Penjelasan: Metode pembayaran harus valid, hanya 'Cash', 'Credit card', atau 'Ewallet'.
result_payment = validator.expect_column_values_to_be_in_set('payment', ['Cash', 'Credit card', 'Ewallet'])
print('Success:', result_payment['success'])
result_payment

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

Success: True


{
  "success": true,
  "result": {
    "element_count": 1000,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

**Analisa/summary hasil validasi:**
- Jumlah data: 1000 baris
- Jumlah nilai di luar 'Cash', 'Credit card', atau 'Ewallet': 0
- Persentase nilai tidak valid: 0%
- Tidak ada nilai yang hilang (missing)
- Semua nilai pada kolom `payment` valid

Hasil ini menunjukkan data metode pembayaran sudah bersih dan konsisten, sehingga dapat digunakan untuk analisis tren pembayaran, strategi promosi, dan pengambilan keputusan bisnis tanpa risiko bias akibat data yang tidak sesuai.

## 5.6 Expectation 6: Kolom `branch` harus berisi salah satu dari [`Alex`, `Giza`, `Cairo`]

Kolom `branch` merepresentasikan cabang supermarket tempat transaksi dilakukan, dengan nilai yang diharapkan hanya 'Alex', 'Giza', atau 'Cairo'. Validasi ini memastikan data lokasi transaksi konsisten dan tidak ada nilai di luar tiga cabang resmi, sehingga analisis performa cabang dan distribusi penjualan dapat dilakukan secara akurat.


In [10]:
# Penjelasan: Branch toko harus valid, hanya 'Alex', 'Giza', atau 'Cairo'.
result_branch = validator.expect_column_values_to_be_in_set('branch', ['Alex', 'Giza', 'Cairo'])
print('Success:', result_branch['success'])
result_branch

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

Success: True


{
  "success": true,
  "result": {
    "element_count": 1000,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

**Analisa/summary hasil validasi:**
- Jumlah data: 1000 baris
- Jumlah nilai di luar 'Alex', 'Giza', atau 'Cairo': 0
- Persentase nilai tidak valid: 0%
- Tidak ada nilai yang hilang (missing)
- Semua nilai pada kolom `branch` valid

Hasil ini menunjukkan data cabang supermarket sudah bersih dan konsisten, sehingga dapat digunakan untuk analisis performa cabang, perencanaan logistik, dan pengambilan keputusan bisnis tanpa risiko bias akibat data yang tidak sesuai.

## 5.7 Expectation 7: Kolom `city` harus bertipe string (object)


In [11]:
# Penjelasan: Nama kota harus bertipe string agar valid untuk analisis.
result_city_type = validator.expect_column_values_to_be_of_type('city', 'object')
print('Success:', result_city_type['success'])
result_city_type

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

Success: True


{
  "success": true,
  "result": {
    "observed_value": "object_"
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

**Analisa/summary hasil validasi:**
- Semua nilai pada kolom `city` bertipe `object` (string) sesuai dengan yang diharapkan.
- Tidak ditemukan nilai dengan tipe data lain.
- Tidak ada error atau exception selama proses validasi.
- Hasil validasi menunjukkan bahwa data kota sudah konsisten dan siap digunakan untuk analisis lokasi, segmentasi pasar, dan visualisasi geografis.

Validasi ini memastikan bahwa kolom `city` dapat diolah dengan fungsi-fungsi string pada analisis data, sehingga tidak akan terjadi error tipe data saat melakukan grouping, filtering, atau visualisasi berdasarkan kota. Data sudah memenuhi standar dan siap digunakan untuk tahap analisis selanjutnya.

In [12]:
# Simpan expectation suite agar dapat digunakan untuk validasi selanjutnya
validator.save_expectation_suite(discard_failed_expectations=False)

# 6. Overall Hasil Analisis dan Review Validasi Data
 
Seluruh proses validasi data pada dataset supermarket sales telah dilakukan menggunakan Great Expectations dengan 7 expectation utama, meliputi keunikan invoice_id, rentang rating, validasi gender, tipe data quantity, validasi payment, validasi branch, dan tipe data city.
 
**Hasil utama dari validasi:**
- Semua expectation berhasil (success: True), menandakan data sudah bersih, konsisten, dan sesuai standar bisnis.
- Tidak ditemukan duplikasi, nilai di luar rentang, atau tipe data yang tidak sesuai pada kolom-kolom utama.
- Data siap digunakan untuk analisis lebih lanjut, seperti segmentasi pelanggan, analisis performa cabang, tren pembayaran, dan visualisasi geografis.
 
**Review:**
- Proses validasi ini memastikan integritas dan kualitas data, sehingga hasil analisis dan insight yang dihasilkan dapat diandalkan untuk pengambilan keputusan bisnis.
- Dengan data yang sudah tervalidasi, risiko error, bias, dan misinterpretasi dapat diminimalisir.
- Notebook ini telah memenuhi seluruh instruksi milestone, baik dari sisi jumlah expectation, penjelasan, analisa, maupun dokumentasi hasil.
 
**Kesimpulan:**
Data supermarket sales sudah siap untuk tahap selanjutnya, seperti integrasi ke Elasticsearch, pembuatan DAG, dan visualisasi di Kibana. Validasi data yang komprehensif menjadi fondasi penting untuk analisis data yang akurat dan actionable.